In [1]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.


In [2]:
import pysnooper
import re

In [20]:
data_path = '../data/output/combined_file/consolidated_full_corpus.txt'
lines = open(data_path, encoding='utf-8').read().split('\n')

In [21]:
new_lines = []
for cnt in range(len(lines)):
    lines[cnt].replace('\u200c', '')
    lines[cnt].replace('\u0020', '')
    sentence = lines[cnt].split('||')
    if len(sentence) < 2:
        print(cnt, lines[cnt], sentence)
#         lines[cnt] += lines[cnt + 1]

#     new_lines.append(lines[cnt])

In [142]:
lines = [line.replace('\u200c', '') for line in lines]
en_text = re.compile('\w')
line_sent = []
error_cnt = 0
for cnt, line in enumerate(lines):
    try:
        line.replace('\u200c', '')
        en_sent, or_sent = line.split('||')
        if re.search(en_text, or_sent): 
            re.sub(en_text, '', en_sent)
            re.sub(en_text, '', or_sent)
        else:
            line_sent.append(en_sent + '||' + or_sent)
    except Exception as error:
        error_cnt += 1
        print(error_cnt, cnt + 1, line, error)
        continue

1 7373 By providing some information about yourself, the audio data you submit to Common Voice will be more useful to Speech not enough values to unpack (expected 2, got 1)
2 7416 You're currently set to NOT receive emails such as goal reminders, my not enough values to unpack (expected 2, got 1)
3 7590 Voice recognition technology is revolutionizing the way we interact with machines, but the currently available systems are expensive and proprietary. Common Voice is part of Mozilla’s initiative to make voice recognition technologies better and more accessible for everyone. Common Voice is a massive global database of donated voices that lets anyone quickly and easily train voice-enabled apps in potentially every language. not enough values to unpack (expected 2, got 1)
4 7591  not enough values to unpack (expected 2, got 1)
5 7593 ଅଧିକ ଲୋକ ବ୍ୟବହାର କରୁଥିବା ଭାଷାଗୁଡ଼ିକ ସହ କମ ଲୋକ କହୁଥିବା ଭାଷାର ମଧ୍ୟ ଆମେ ସ୍ୱର ସଂଗ୍ରହ କରୁଛୁ । ଧ୍ୱନିଗୁଡ଼ିକର ବିବିଧ ତଥ୍ୟଭଣ୍ଡାରକୁ ପ୍ରକାଶ କରିବାକୁ ଡେଭେଲପର, ଉଦ୍ୟମି ଏବଂ ସ

110 10874 Add-ons are applications that let you personalize &brandShortName; with not enough values to unpack (expected 2, got 1)
111 10875   extra functionality or style. Try a time-saving sidebar, a weather notifier, or a themed look to make &brandShortName; not enough values to unpack (expected 2, got 1)
112 10984 Importing history not enough values to unpack (expected 2, got 1)
113 11071 The following add-ons are not compatible with this version of not enough values to unpack (expected 2, got 1)
114 11089 Voice is natural, voice is human. That’s why we’re excited about creating usable voice technology not enough values to unpack (expected 2, got 1)
115 11090 for our machines. But to create voice systems, developers need an extremely large amount of voice not enough values to unpack (expected 2, got 1)
116 11202 It is not possible to check for updates to incompatible add-ons not enough values to unpack (expected 2, got 1)
117 11203                                            because 

In [62]:
eng_sent = []
fra_sent = []
eng_chars = set()
fra_chars = set()
nb_samples = 1000

# Process english and french sentences
for line in range(nb_samples):
    eng_line = str(lines[line]).split('||')[0]
    # Append '\t' for start of the sentence and '\n' to signify end of the sentence
#     print(line, lines[line])
    fra_line = '\t' + str(lines[line]).split('||')[1] + '\n'
    eng_sent.append(eng_line)
    fra_sent.append(fra_line)
    
    for ch in eng_line:
        if (ch not in eng_chars):
            eng_chars.add(ch)
            
    for ch in fra_line:
        if (ch not in fra_chars):
            fra_chars.add(ch)

In [63]:
eng_sentences, fra_sentences = eng_sent, fra_sent

In [64]:
fra_chars = sorted(list(fra_chars))
eng_chars = sorted(list(eng_chars))

In [65]:
# dictionary to index each english character - key is index and value is english character
eng_index_to_char_dict = {}

# dictionary to get english character given its index - key is english character and value is index
eng_char_to_index_dict = {}

for k, v in enumerate(eng_chars):
    eng_index_to_char_dict[k] = v
    eng_char_to_index_dict[v] = k

In [67]:
fra_chars

['\t',
 '\n',
 ' ',
 '(',
 ')',
 '-',
 '/',
 '0',
 '1',
 '2',
 '3',
 '6',
 'A',
 'B',
 'C',
 'D',
 'G',
 'I',
 'L',
 'M',
 'P',
 'R',
 'S',
 'T',
 'U',
 'V',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'ଁ',
 'ଂ',
 'ଃ',
 'ଅ',
 'ଆ',
 'ଇ',
 'ଉ',
 'ଋ',
 'ଏ',
 'ଓ',
 'କ',
 'ଖ',
 'ଗ',
 'ଘ',
 'ଙ',
 'ଚ',
 'ଛ',
 'ଜ',
 'ଞ',
 'ଟ',
 'ଠ',
 'ଡ',
 'ଢ',
 'ଣ',
 'ତ',
 'ଥ',
 'ଦ',
 'ଧ',
 'ନ',
 'ପ',
 'ଫ',
 'ବ',
 'ଭ',
 'ମ',
 'ଯ',
 'ର',
 'ଲ',
 'ଳ',
 'ଵ',
 'ଶ',
 'ଷ',
 'ସ',
 'ହ',
 '଼',
 'ା',
 'ି',
 'ୀ',
 'ୁ',
 'ୂ',
 'ୃ',
 'େ',
 'ୈ',
 'ୋ',
 'ୌ',
 '୍',
 'ୟ',
 'ୱ',
 '\u200d']

In [45]:
# dictionary to index each french character - key is index and value is french character
fra_index_to_char_dict = {}

# dictionary to get french character given its index - key is french character and value is index
fra_char_to_index_dict = {}
for k, v in enumerate(fra_chars):
    fra_index_to_char_dict[k] = v
    fra_char_to_index_dict[v] = k

In [49]:
max_len_eng_sent = max([len(line) for line in eng_sentences])
max_len_fra_sent = max([len(line) for line in fra_sentences])
print(max_len_eng_sent, max_len_fra_sent)

45 61


In [50]:
tokenized_eng_sentences = np.zeros(shape = (nb_samples,max_len_eng_sent,len(eng_chars)), dtype='float32')
tokenized_fra_sentences = np.zeros(shape = (nb_samples,max_len_fra_sent,len(fra_chars)), dtype='float32')
target_data = np.zeros((nb_samples, max_len_fra_sent, len(fra_chars)),dtype='float32')

In [51]:
# Vectorize the english and french sentences

for i in range(nb_samples):
    for k,ch in enumerate(eng_sent[i]):
        tokenized_eng_sentences[i,k,eng_char_to_index_dict[ch]] = 1
        
    for k,ch in enumerate(fra_sent[i]):
        tokenized_fra_sentences[i,k,fra_char_to_index_dict[ch]] = 1

        # decoder_target_data will be ahead by one timestep and will not include the start character.
        if k > 0:
            target_data[i,k-1,fra_char_to_index_dict[ch]] = 1

In [52]:
# Encoder model
encoder_input = Input(shape=(None,len(eng_chars)))
encoder_LSTM = LSTM(256,return_state = True)
encoder_outputs, encoder_h, encoder_c = encoder_LSTM (encoder_input)
encoder_states = [encoder_h, encoder_c]

W1123 20:40:36.937596 10244 deprecation_wrapper.py:119] From c:\users\soumendraksahoo\appdata\local\conda\conda\envs\wikicorpus\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1123 20:40:37.046956 10244 deprecation_wrapper.py:119] From c:\users\soumendraksahoo\appdata\local\conda\conda\envs\wikicorpus\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1123 20:40:37.078231 10244 deprecation_wrapper.py:119] From c:\users\soumendraksahoo\appdata\local\conda\conda\envs\wikicorpus\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [53]:
# Decoder model

decoder_input = Input(shape=(None,len(fra_chars)))
decoder_LSTM = LSTM(256,return_sequences=True, return_state = True)
decoder_out, _ , _ = decoder_LSTM(decoder_input, initial_state=encoder_states)
decoder_dense = Dense(len(fra_chars),activation='softmax')
decoder_out = decoder_dense (decoder_out)

In [55]:
model = Model(inputs=[encoder_input, decoder_input],outputs=[decoder_out])

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit(x=[tokenized_eng_sentences,tokenized_fra_sentences], 
          y=target_data,
          batch_size=64,
          epochs=10,
          validation_split=0.2)

Train on 800 samples, validate on 200 samples
Epoch 1/10
800/800 [==============================] - 7s 9ms/step - loss: 0.7459 - val_loss: 0.6969
Epoch 2/10
800/800 [==============================] - 7s 8ms/step - loss: 0.7153 - val_loss: 0.6819
Epoch 3/10
800/800 [==============================] - 7s 8ms/step - loss: 0.7012 - val_loss: 0.6729
Epoch 4/10
800/800 [==============================] - 6s 8ms/step - loss: 0.6922 - val_loss: 0.6610
Epoch 5/10
800/800 [==============================] - 6s 7ms/step - loss: 0.6798 - val_loss: 0.6530
Epoch 6/10
800/800 [==============================] - 6s 7ms/step - loss: 0.6689 - val_loss: 0.6497
Epoch 7/10
800/800 [==============================] - 6s 7ms/step - loss: 0.6570 - val_loss: 0.6407
Epoch 8/10
800/800 [==============================] - 6s 7ms/step - loss: 0.6489 - val_loss: 0.6249
Epoch 9/10
800/800 [==============================] - 6s 8ms/step - loss: 0.6384 - val_loss: 0.6217
Epoch 10/10
800/800 [==============================] -

In [56]:
# Inference models for testing

# Encoder inference model
encoder_model_inf = Model(encoder_input, encoder_states)

# Decoder inference model
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_input_states = [decoder_state_input_h, decoder_state_input_c]

decoder_out, decoder_h, decoder_c = decoder_LSTM(decoder_input, 
                                                 initial_state=decoder_input_states)

decoder_states = [decoder_h , decoder_c]

decoder_out = decoder_dense(decoder_out)

decoder_model_inf = Model(inputs=[decoder_input] + decoder_input_states,
                          outputs=[decoder_out] + decoder_states )

In [57]:
def decode_seq(inp_seq):
    
    # Initial states value is coming from the encoder 
    states_val = encoder_model_inf.predict(inp_seq)
    
    target_seq = np.zeros((1, 1, len(fra_chars)))
    target_seq[0, 0, fra_char_to_index_dict['\t']] = 1
    
    translated_sent = ''
    stop_condition = False
    
    while not stop_condition:
        
        decoder_out, decoder_h, decoder_c = decoder_model_inf.predict(x=[target_seq] + states_val)
        
        max_val_index = np.argmax(decoder_out[0,-1,:])
        sampled_fra_char = fra_index_to_char_dict[max_val_index]
        translated_sent += sampled_fra_char
        
        if ( (sampled_fra_char == '\n') or (len(translated_sent) > max_len_fra_sent)) :
            stop_condition = True
        
        target_seq = np.zeros((1, 1, len(fra_chars)))
        target_seq[0, 0, max_val_index] = 1
        
        states_val = [decoder_h, decoder_c]
        
    return translated_sent

In [58]:
for seq_index in range(10):
    inp_seq = tokenized_eng_sentences[seq_index:seq_index+1]
    translated_sent = decode_seq(inp_seq)
    print('-')
    print('Input sentence:', eng_sent[seq_index])
    print('Decoded sentence:', translated_sent)

-
Input sentence: video ad
Decoded sentence: ସ୍ରାପ୍ କ୍ର୍

-
Input sentence: video ad
Decoded sentence: ସ୍ରାପ୍ କ୍ର୍

-
Input sentence: privacy
Decoded sentence: ସ୍ରାପ୍ କ୍ର୍

-
Input sentence: unread
Decoded sentence: ସ୍ରାପ୍ କ୍ର୍

-
Input sentence: recharge shop
Decoded sentence: ସ୍ରାପ୍ କ୍ର୍

-
Input sentence: preset study
Decoded sentence: ସ୍ରାପ୍ କ୍ର୍

-
Input sentence: default ad
Decoded sentence: ସ୍ରାପ୍ କ୍ର୍

-
Input sentence: weighted split
Decoded sentence: ସ୍ରାପ୍ କ୍ର୍

-
Input sentence: Ads Manager
Decoded sentence: ସ୍ରାପ୍ କ୍ର୍

-
Input sentence: customize
Decoded sentence: ସ୍ରାପ୍ କ୍ର୍



In [59]:
fra_sent

['\tଭିଡିଓ ବିଜ୍ଞାପନ\n',
 '\tଭିଡିଓ ବିଜ୍ଞାପନ\n',
 '\tଗୋପନୀୟତା\n',
 '\tଅପଠିତ\n',
 '\tରିଚାର୍ଜ୍ ଦୋକାନ\n',
 '\tପ୍ରିସେଟ୍ ଅଧ୍ୟୟନ\n',
 '\tଡିଫଲ୍ଟ ବିଜ୍ଞାପନ\n',
 '\tଗୁରୁତ୍ୱପୂର୍ଣ୍ଣ ବିଭାଜନ\n',
 '\tବିଜ୍ଞାପନ ପରିଚାଳକ\n',
 '\tଅନୁକୂଳ କରାଇବା\n',
 '\tସୃଜନଶୀଳ ଧାରଣା\n',
 '\tସଙ୍ଗୀତ ପାଇଁ Rights Manager\n',
 '\tଅଙ୍କନ କରିବା\n',
 '\tସୁରକ୍ଷିତ ବ୍ରାଉଜିଂ\n',
 '\tନିଜର ବୋଲି ଦାବି ହୋଇଥିବା ଟ୍ୟାଗ୍\n',
 '\tରୋଲ୍ ଓଭର୍\n',
 '\tଅନୁସରଣ କରନ୍ତୁ ନାହିଁ\n',
 '\tପୃଷ୍ଠା ପସନ୍ଦ\n',
 '\tସ୍ଥାନ ଗ୍ରାଫ\n',
 '\tସ୍ନୁଜ୍\n',
 '\tଚାଟ୍\n',
 '\tସ୍ପର୍ଶ\n',
 '\tପ୍ରାୟୋଜିତ\n',
 '\tଲାଇଭ୍ ଷ୍ଟ୍ରିମ୍\n',
 '\tଲଗଇନ୍ ସୂଚନା\n',
 '\tବ୍ୟାପକତା\n',
 '\tପେଲିବା/ପୁଶ୍\n',
 '\tଲକ୍ ଇନ୍\n',
 '\tଆଉଟବାଉଣ୍ଡ କ୍ଲିକଥ୍ରୁ ମୂଲ୍ୟ\n',
 '\tକ୍ରେତା\n',
 '\tଲାଇଭ୍ କ୍ରିଏଟିଭ୍ କିଟ୍\n',
 '\tୱାଲେଟ୍\n',
 '\tଦୃଷ୍ଟିପାତ\n',
 '\tକ୍ୟାମେରା ପ୍ରଭାବ\n',
 '\tପ୍ରଚାର\n',
 '\tଡିଫଲ୍ଟ ଗୋଷ୍ଠୀ\n',
 '\tକାଟାଲଗ୍ ବିଭାଗ\n',
 '\tଆଖି ଠାର\n',
 '\tସାମାଜିକ ପ୍ରଭାବ\n',
 '\tମୂଳ ତାଲିକାକରଣ ସମାହାର\n',
 '\tଫ୍ଲିକ୍\n',
 '\tକ୍ୟାପଚର୍\n',
 '\tବିଜ୍ଞାପନ ରୋଜଗାର\n',
 '\tଅପ୍ରକାଶିତ ପୃଷ୍ଠା ପୋଷ୍ଟ୍\n',
 '\tପ୍ଲେସମେଣ୍ଟ\n',
 '\tସମୀକ୍ଷାରେ ବର୍ଷ\n'